# Demo of comparison of SCF with BinarySearch  (NeurIPS'23)

## 1. Introduction

BinarySearch, also known as Individual DP-SGD (IDP-SGD) with the Sample mechanism, is a binary-search-based approach aiming for finding a suitable sampling probability $q$ within the range of $[0,1]$ for a target privacy budget $\varepsilon$. In contrast to our setup where each individual point (record) in the training dataset are assigned a personalized, non-uniform privacy budget $\varepsilon$, the authors of [1] consider a simplified scenario where points are split into **privacy groups** and each group is assigned a different privacy budget.

<img src="./figures/binary_pseudocode.png"/>

Ref:
[1] Algorithm 1 in the NeurIPS'23 paper [Have it your way: Individualized Privacy Assignment for DP-SGD](https://openreview.net/forum?id=XXPzBhOs4f)

## 2. Differences in terms of the privacy budget distributions

Although BinarySearch is also built on top of the idea of integrating non-uniform sampling into the classical DPSGD algorithm, it is, however, computationally demanding for our settings where records' privacy budgets are distributed continuously (e.g., follow a Gaussian or Pareto distribution) and cover a range of values. 

To illustrate this, we conduct a toy experiment involving a subset of 1,000 records (drawn from the MNIST dataset) and the objective is to determine a proper sampling probability for each record under the following three cases:
- `Group-3`: 3 unevenly sized privacy groups with budgets [1., 2., 3.]
- `Group-100`: 100 evenly sized privacy groups with budgets [1, 1.05, 1.1, ..., 5.9, 5.95]
- `Individual-1000`: each individual record are assigned a personalized budget that is randomly drawn a BoundedMixGauss distribution ranging from 0.1 to 10.0. 

In [1]:
import numpy as np
import os
import sys
sys.path.append("..")

from torch.utils.data import DataLoader, Subset
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor

project_abspath = os.path.abspath(os.path.join(os.getcwd(),".."))
data_path = os.path.join(project_abspath, "datasets/mnist")
train_data = MNIST(data_path, train=True, download=True, transform=Compose([ToTensor(), Normalize(0.5, 0.5)]))
test_data = MNIST(data_path, train=False, download=True, transform=Compose([ToTensor(), Normalize(0.5, 0.5)]))

train_indices = list(range(len(train_data)))
np.random.shuffle(train_indices)
train_data = Subset(train_data, train_indices[:1000])
n_data = len(train_data)
n_data

1000

In [2]:
""" Prepare personalized privacy budgets """
def assign_budgets(n_data: int, budgets: list, ratios: list) -> np.ndarray:
    # ref: https://openreview.net/forum?id=XXPzBhOs4f. Implemented by Boenisch et al. (2023)
    assert all(np.array(budgets) > 0), 'Privacy budgets must be positive!'
    assert len(
        np.unique(budgets)) == len(budgets), 'Budgets must be different!'
    assert math.fsum(ratios) == 1, f'Ratios sum up to {math.fsum(ratios)}, ' \
                                   f'but they must sum up to 1!'
    assert len(budgets) == len(ratios), 'Numbers of budgets and ratios must ' \
                                        'be equal!'
    indices = np.arange(n_data)
    pp_budgets = np.zeros(n_data) # per-point budgets
    n_groups = len(ratios)
    # randomly assign budgets
    for group in range(n_groups - 1):
        idx = np.random.choice(a=indices,
                                size=int(round(n_data * ratios[group])),
                                replace=False)
        pp_budgets[idx] = budgets[group]
        indices = np.setdiff1d(indices, idx)
    pp_budgets[pp_budgets == 0] = budgets[-1]
    return pp_budgets

def get_target_epsilons(setup):
    if setup == "group3":
        ### Group-3
        budgets = [1., 2., 3.]
        ratios = [0.54, 0.37, 0.09]
        target_epsilons = assign_budgets(n_data, budgets, ratios)
        print("target_epsilons (unique): ", np.unique(target_epsilons))

    elif setup == "group100":
        ### Group-100
        budgets=np.linspace(1,5.95,100) # 100 evenly sized privacy groups with budgets [1, 1.05, 1.1, ..., 5.9, 5.95]
        ratios=np.ones(100) * 0.01
        target_epsilons = assign_budgets(n_data, budgets, ratios)
        print("target_epsilons (unique): ", np.unique(target_epsilons))

    else:
        ### Individual-1000
        MIN_EPSILON, MAX_EPSILON = 0.1, 10.0
        BoundedFunc = lambda values: np.array([min(max(x, MIN_EPSILON), MAX_EPSILON) for x in values])
        target_epsilons = BoundedFunc(MixGauss([0.7,0.2,0.1], [(1.0, 0.1), (2.0, 0.2), (5.0, 0.5)], n_data))
        print("target_epsilons: ", target_epsilons)
    return target_epsilons

In [3]:
import math
import pandas as pd
import time
import torch
import torch.optim as optim
from myopacus import PrivacyEngine
from myopacus.accountants.rpdp_utils import MixGauss

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# seed = 42
# np.random.seed(seed)
# torch.manual_seed(seed)

# Hyper-parameters for **both** methods
NUM_STEPS = 100
LR_DP = 0.1
TARGET_DELTA = 1e-3
MAX_GRAD_NORM = 1.0

## 3. Experimental setups

In this demo, we consider a simplified personalized privacy scenario *ThreeLevels*, where the percentage of records with $\varepsilon_1$, $\varepsilon_2$, and $\varepsilon_3$ is 70\%, 20\%, and 10\%, respectively. 

In [4]:
from torch.utils.data import DataLoader
""" Prepare data_loader """
train_loader = DataLoader(train_data, batch_size=len(train_data))
test_loader = DataLoader(test_data, batch_size=len(test_data))

In [5]:
from datasets.fed_mnist import BaselineModel, BaselineLoss, metric
""" Prepare model, loss, optim """
model = BaselineModel.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR_DP, momentum=0)
criterion = BaselineLoss()

## 4. DP-SGD with BinarySearch / SCF

In [7]:
from myopacus.accountants import create_accountant
from myopacus.accountants.rpdp_utils import PrivCostEstimator
from myopacus.accountants.utils import get_noise_multiplier
# Authors : Boenisch et al. (2023), Ref: Supplementary material in https://openreview.net/forum?id=XXPzBhOs4f
MAX_SIGMA = 1e6
MIN_Q = 1e-5
MAX_Q = 1.0

# idp_sgd/opacus/opacus/accountants/utils.py
def get_sample_rate(
    target_epsilon: float,
    target_delta: float,
    noise_multiplier: float,
    steps: int,
    accountant: str = "rdp",
    precision: float = 0.001,
    **kwargs,
) -> float:
    r"""
    Computes via binary search the sampling frequency q to reach a total budget
    of (target_epsilon, target_delta) at the end of epochs, with a given
    noise_multiplier.
    Args:
        target_epsilon: the privacy budget's epsilon
        target_delta: the privacy budget's delta
        noise_multiplier: relation between noise std and clipping threshold
        steps: number of steps to run
        accountant: accounting mechanism used to estimate epsilon
        precision: relation between limits of binary search interval
    Returns:
        The sampling frequency q to ensure privacy budget of
        (target_epsilon, target_delta)
    """
    # print("target_epsilon: ", target_epsilon)
    accountant = create_accountant(mechanism=accountant)
    q_low, q_high = MIN_Q, MAX_Q
    accountant.history = [(noise_multiplier, q_low, steps)]
    eps_low = accountant.get_epsilon(delta=target_delta, **kwargs)
    # print("q_low: ", q_low, " eps_low: ", eps_low)
    if eps_low > target_epsilon:
        raise ValueError("The privacy budget is too low.")
    accountant.history = [(noise_multiplier, q_high, steps)]
    eps_high = accountant.get_epsilon(delta=target_delta, **kwargs)
    # print("q_high: ", q_high, " eps_high: ", eps_high)
    while eps_high < 0:     # decrease q_high whenever a numerical error happens
        q_high *= 0.9
        accountant.history = [(noise_multiplier, q_high, steps)]
        eps_high = accountant.get_epsilon(delta=target_delta, **kwargs)
        # print("q_high: ", q_high, " eps_high: ", eps_high)

    while q_low / q_high < 1 - precision:
        q = (q_low + q_high) / 2
        accountant.history = [(noise_multiplier, q, steps)]
        eps = accountant.get_epsilon(delta=target_delta, **kwargs)
        if eps < target_epsilon:
            q_low = q
        else:
            q_high = q

    return q_low

def binary_search(noise_multiplier, target_epsilons, target_delta, steps):
    weights = []
    for i, budget in enumerate(np.sort(np.unique(target_epsilons))):
        sample_rate = get_sample_rate(
            target_epsilon=budget,
            target_delta=target_delta,
            noise_multiplier=noise_multiplier,
            steps=steps, # total sgd iterations
            accountant="rdp"
        )
        weights.append(sample_rate)
        
    sample_rates = np.zeros(len(target_epsilons))
    for i, budget in enumerate(np.sort(np.unique(target_epsilons))):
        sample_rates[target_epsilons == budget] = weights[i]
    return sample_rates

def simulation_curvefitting(noise_multiplier, target_epsilons, target_delta, steps):
    pce = PrivCostEstimator(
        noise_multiplier = noise_multiplier, 
        steps = steps, 
        delta = target_delta
    )
    fit_fn = pce.get_sample_rate_estimator()
    
    sample_rates = np.zeros(len(target_epsilons))
    for i, budget in enumerate(np.sort(np.unique(target_epsilons))):
        sample_rates[target_epsilons == budget] = fit_fn(budget)
    return sample_rates

nm_list = []
results = {
    "group3": {"binary": [], "scf":[] }, 
    "group100": {"binary": [], "scf":[] },
    "individual1000": {"binary": [], "scf":[] },
}

### Average runtimes: Group-3

Note: ~1.69 seconds per trial for BinarySearch and ~13.11 seconds per trial for SCF, total 5 trials

In [8]:
case = "group3"
target_epsilons = get_target_epsilons(case)
noise_multiplier = get_noise_multiplier(
    target_epsilon=max(target_epsilons),
    target_delta=TARGET_DELTA,
    sample_rate=1.0,
    steps=NUM_STEPS,
    accountant="rdp"
)
t4binary, t4scf = [], []
for _ in range(5): 
    start = time.time()
    sample_rates = binary_search(noise_multiplier, target_epsilons, TARGET_DELTA, NUM_STEPS)
    elapsed_time = time.time() - start
    t4binary.append(elapsed_time)
    print(noise_multiplier, np.unique(sample_rates))
    print(elapsed_time)
    
    start = time.time()
    sample_rates = simulation_curvefitting(noise_multiplier, target_epsilons, TARGET_DELTA, NUM_STEPS)
    elapsed_time = time.time() - start
    t4scf.append(elapsed_time)
    print(noise_multiplier, np.unique(sample_rates))
    print(elapsed_time)

results[case]["binary"].append(t4binary)
results[case]["scf"].append(t4scf)
print(f"The average runtime of BinarySearch in the case of `{case}`: {np.mean(t4binary)}")
print(f"The average runtime of SCF in the case of `{case}`: {np.mean(t4scf)}")
print("NOTE: this does not account for the time required for model training.")

target_epsilons (unique):  [1. 2. 3.]


/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


11.484375 [0.39209592 0.71094039 0.99902345]
2.378804922103882
The R-Squared value of the best-fit curve : 0.9998557975952399
11.484375 [0.39089506 0.71363807 1.        ]
12.865214347839355


/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


11.484375 [0.39209592 0.71094039 0.99902345]
1.5309197902679443
The R-Squared value of the best-fit curve : 0.9998557975952399
11.484375 [0.39089506 0.71363807 1.        ]
13.264188766479492


...

11.484375 [0.39209592 0.71094039 0.99902345]
1.5155136585235596
The R-Squared value of the best-fit curve : 0.9998557975952399
11.484375 [0.39089506 0.71363807 1.        ]
13.166598081588745


/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


11.484375 [0.39209592 0.71094039 0.99902345]
1.5061767101287842
The R-Squared value of the best-fit curve : 0.9998557975952399
11.484375 [0.39089506 0.71363807 1.        ]
13.103019952774048


/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


11.484375 [0.39209592 0.71094039 0.99902345]
1.5133376121520996
The R-Squared value of the best-fit curve : 0.9998557975952399
11.484375 [0.39089506 0.71363807 1.        ]
13.154590606689453
The average runtime of BinarySearch in the case of `group3`: 1.688950538635254
The average runtime of SCF in the case of `group3`: 13.11072235107422
NOTE: this does not account for the time required for model training.


### Average runtimes: Group-100

Note: ~52.50 seconds per trial for BinarySearch and ~13.31 seconds per trial for SCF, total 5 trials

In [9]:
case = "group100"
target_epsilons = get_target_epsilons(case)
noise_multiplier = get_noise_multiplier(
    target_epsilon=max(target_epsilons),
    target_delta=TARGET_DELTA,
    sample_rate=1.0,
    steps=NUM_STEPS,
    accountant="rdp"
)
t4binary, t4scf = [], []
for _ in range(5): 
    start = time.time()
    sample_rates = binary_search(noise_multiplier, target_epsilons, TARGET_DELTA, NUM_STEPS)
    elapsed_time = time.time() - start
    t4binary.append(elapsed_time)
    print(elapsed_time)
    
    start = time.time()
    sample_rates = simulation_curvefitting(noise_multiplier, target_epsilons, TARGET_DELTA, NUM_STEPS)
    elapsed_time = time.time() - start
    t4scf.append(elapsed_time)
    print(elapsed_time)

results[case]["binary"].append(t4binary)
results[case]["scf"].append(t4scf)
print(f"The average runtime of BinarySearch in the case of `{case}`: {np.mean(t4binary)}")
print(f"The average runtime of SCF in the case of `{case}`: {np.mean(t4scf)}")
print("NOTE: this does not account for the time required for model training.")

target_epsilons (unique):  [1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45 1.5  1.55 1.6  1.65
 1.7  1.75 1.8  1.85 1.9  1.95 2.   2.05 2.1  2.15 2.2  2.25 2.3  2.35
 2.4  2.45 2.5  2.55 2.6  2.65 2.7  2.75 2.8  2.85 2.9  2.95 3.   3.05
 3.1  3.15 3.2  3.25 3.3  3.35 3.4  3.45 3.5  3.55 3.6  3.65 3.7  3.75
 3.8  3.85 3.9  3.95 4.   4.05 4.1  4.15 4.2  4.25 4.3  4.35 4.4  4.45
 4.5  4.55 4.6  4.65 4.7  4.75 4.8  4.85 4.9  4.95 5.   5.05 5.1  5.15
 5.2  5.25 5.3  5.35 5.4  5.45 5.5  5.55 5.6  5.65 5.7  5.75 5.8  5.85
 5.9  5.95]


/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


6.6015625 [0.22241989 0.23194127 0.2417068  0.25098405 0.26001717 0.26953855
 0.27905994 0.28809306 0.29712617 0.30567101 0.3151924  0.32446965
 0.33350276 0.34229173 0.35108071 0.35938141 0.3676821  0.37671522
 0.38599247 0.39478144 0.40357042 0.41235939 0.42066009 0.42896079
 0.43701735 0.44507391 0.45288633 0.46069875 0.46948773 0.47852084
 0.48730981 0.49561051 0.50439949 0.51270019 0.52100088 0.52881331
 0.537114   0.54492643 0.55273885 0.56055127 0.56787542 0.57568784
 0.58301198 0.59033613 0.597172   0.60449614 0.61133201 0.62012099
 0.62842168 0.63721066 0.64551136 0.65381206 0.66211275 0.67041345
 0.67822587 0.6860383  0.69385072 0.70166314 0.70947556 0.71728798
 0.72461213 0.73242455 0.7397487  0.74707284 0.75439699 0.76123286
 0.768557   0.77539287 0.78271702 0.78955289 0.79638875 0.80322462
 0.81006049 0.81689636 0.82373223 0.83007982 0.83691569 0.84326329
 0.85009916 0.85644675 0.86279434 0.86914193 0.87548953 0.88183712
 0.88818471 0.8945323  0.90039162 0.90820404 0.91601

/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


6.6015625 [0.22241989 0.23194127 0.2417068  0.25098405 0.26001717 0.26953855
 0.27905994 0.28809306 0.29712617 0.30567101 0.3151924  0.32446965
 0.33350276 0.34229173 0.35108071 0.35938141 0.3676821  0.37671522
 0.38599247 0.39478144 0.40357042 0.41235939 0.42066009 0.42896079
 0.43701735 0.44507391 0.45288633 0.46069875 0.46948773 0.47852084
 0.48730981 0.49561051 0.50439949 0.51270019 0.52100088 0.52881331
 0.537114   0.54492643 0.55273885 0.56055127 0.56787542 0.57568784
 0.58301198 0.59033613 0.597172   0.60449614 0.61133201 0.62012099
 0.62842168 0.63721066 0.64551136 0.65381206 0.66211275 0.67041345
 0.67822587 0.6860383  0.69385072 0.70166314 0.70947556 0.71728798
 0.72461213 0.73242455 0.7397487  0.74707284 0.75439699 0.76123286
 0.768557   0.77539287 0.78271702 0.78955289 0.79638875 0.80322462
 0.81006049 0.81689636 0.82373223 0.83007982 0.83691569 0.84326329
 0.85009916 0.85644675 0.86279434 0.86914193 0.87548953 0.88183712
 0.88818471 0.8945323  0.90039162 0.90820404 0.91601

...

6.6015625 [0.22241989 0.23194127 0.2417068  0.25098405 0.26001717 0.26953855
 0.27905994 0.28809306 0.29712617 0.30567101 0.3151924  0.32446965
 0.33350276 0.34229173 0.35108071 0.35938141 0.3676821  0.37671522
 0.38599247 0.39478144 0.40357042 0.41235939 0.42066009 0.42896079
 0.43701735 0.44507391 0.45288633 0.46069875 0.46948773 0.47852084
 0.48730981 0.49561051 0.50439949 0.51270019 0.52100088 0.52881331
 0.537114   0.54492643 0.55273885 0.56055127 0.56787542 0.57568784
 0.58301198 0.59033613 0.597172   0.60449614 0.61133201 0.62012099
 0.62842168 0.63721066 0.64551136 0.65381206 0.66211275 0.67041345
 0.67822587 0.6860383  0.69385072 0.70166314 0.70947556 0.71728798
 0.72461213 0.73242455 0.7397487  0.74707284 0.75439699 0.76123286
 0.768557   0.77539287 0.78271702 0.78955289 0.79638875 0.80322462
 0.81006049 0.81689636 0.82373223 0.83007982 0.83691569 0.84326329
 0.85009916 0.85644675 0.86279434 0.86914193 0.87548953 0.88183712
 0.88818471 0.8945323  0.90039162 0.90820404 0.91601

/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


6.6015625 [0.22241989 0.23194127 0.2417068  0.25098405 0.26001717 0.26953855
 0.27905994 0.28809306 0.29712617 0.30567101 0.3151924  0.32446965
 0.33350276 0.34229173 0.35108071 0.35938141 0.3676821  0.37671522
 0.38599247 0.39478144 0.40357042 0.41235939 0.42066009 0.42896079
 0.43701735 0.44507391 0.45288633 0.46069875 0.46948773 0.47852084
 0.48730981 0.49561051 0.50439949 0.51270019 0.52100088 0.52881331
 0.537114   0.54492643 0.55273885 0.56055127 0.56787542 0.57568784
 0.58301198 0.59033613 0.597172   0.60449614 0.61133201 0.62012099
 0.62842168 0.63721066 0.64551136 0.65381206 0.66211275 0.67041345
 0.67822587 0.6860383  0.69385072 0.70166314 0.70947556 0.71728798
 0.72461213 0.73242455 0.7397487  0.74707284 0.75439699 0.76123286
 0.768557   0.77539287 0.78271702 0.78955289 0.79638875 0.80322462
 0.81006049 0.81689636 0.82373223 0.83007982 0.83691569 0.84326329
 0.85009916 0.85644675 0.86279434 0.86914193 0.87548953 0.88183712
 0.88818471 0.8945323  0.90039162 0.90820404 0.91601

/home/junxu/rpdp_fl/myopacus/accountants/analysis/rdp.py:355: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


6.6015625 [0.22241989 0.23194127 0.2417068  0.25098405 0.26001717 0.26953855
 0.27905994 0.28809306 0.29712617 0.30567101 0.3151924  0.32446965
 0.33350276 0.34229173 0.35108071 0.35938141 0.3676821  0.37671522
 0.38599247 0.39478144 0.40357042 0.41235939 0.42066009 0.42896079
 0.43701735 0.44507391 0.45288633 0.46069875 0.46948773 0.47852084
 0.48730981 0.49561051 0.50439949 0.51270019 0.52100088 0.52881331
 0.537114   0.54492643 0.55273885 0.56055127 0.56787542 0.57568784
 0.58301198 0.59033613 0.597172   0.60449614 0.61133201 0.62012099
 0.62842168 0.63721066 0.64551136 0.65381206 0.66211275 0.67041345
 0.67822587 0.6860383  0.69385072 0.70166314 0.70947556 0.71728798
 0.72461213 0.73242455 0.7397487  0.74707284 0.75439699 0.76123286
 0.768557   0.77539287 0.78271702 0.78955289 0.79638875 0.80322462
 0.81006049 0.81689636 0.82373223 0.83007982 0.83691569 0.84326329
 0.85009916 0.85644675 0.86279434 0.86914193 0.87548953 0.88183712
 0.88818471 0.8945323  0.90039162 0.90820404 0.91601

### Average runtimes: individual-1000

Note: ~600 seconds (10 minutes) per trial for BinarySearch and ~14.09 seconds per trial for SCF, total 5 trials

In [12]:
case = "individual1000"
target_epsilons = get_target_epsilons(case)
noise_multiplier = get_noise_multiplier(
    target_epsilon=max(target_epsilons),
    target_delta=TARGET_DELTA,
    sample_rate=1.0,
    steps=NUM_STEPS,
    accountant="rdp"
)
t4binary, t4scf = [], []
for _ in range(5):
    start = time.time()
    sample_rates = binary_search(noise_multiplier, target_epsilons, TARGET_DELTA, NUM_STEPS)
    elapsed_time = time.time() - start
    t4binary.append(elapsed_time)
    print(elapsed_time)
    
    start = time.time()
    sample_rates = simulation_curvefitting(noise_multiplier, target_epsilons, TARGET_DELTA, NUM_STEPS)
    elapsed_time = time.time() - start
    t4scf.append(elapsed_time)
    print(elapsed_time)

results[case]["binary"].append(t4binary)
results[case]["scf"].append(t4scf)
print(f"The average runtime of BinarySearch in the case of `{case}`: {np.mean(t4binary)}")
print(f"The average runtime of SCF in the case of `{case}`: {np.mean(t4scf)}")
print("NOTE: this does not account for the time required for model training.")

target_epsilons:  [1.17460906 4.73859959 1.1020445  5.27837184 1.0052426  2.12136061
 0.85463801 5.86977224 0.76118377 1.06296011 0.87029777 2.01790086
 2.19348747 2.08106585 1.02779054 1.19118314 4.51879865 1.05338391
 2.17034642 0.98364093 0.99736547 0.97173335 0.94721534 0.98775666
 0.87092172 0.96312198 0.97891208 0.91667554 1.20476463 1.88963457
 4.41112022 5.6611948  1.00363896 1.89168908 1.06936498 2.1410098
 1.02874308 0.99955606 0.92729526 0.95278764 0.95191114 1.05795156
 1.0669313  1.05023191 0.96428849 1.01554636 1.02860809 1.02087249
 1.06595195 1.08650178 1.00121494 5.90210224 5.46677337 2.01414271
 0.97637164 1.05659917 0.9869484  0.95848748 1.63718166 1.01551737
 5.67495358 1.20561819 1.03398387 1.91005568 1.00236391 0.9360263
 1.17998924 1.06676004 0.82212859 5.38011254 1.07494873 0.94344511
 0.97368486 0.89981089 1.05086759 5.05603987 1.07982819 1.02176369
 0.89060988 5.76199794 0.97816945 4.25422345 5.47944105 2.15712843
 0.93875368 1.10210371 1.02726046 0.94285215 1

# Conclusion

Our experiment results demonstrate that executing the entire searching process via BinarySearch sequentially would take more than 64 minutes on average across 5 trials\footnote{}. In contrast, SCF achieves both computing the best-fit model and estimating the sampling probability for all records in only around 8 seconds. 

This represents a substantial improvement in efficiency compared to BinarySearch. Note that we did not evaluate BinarySearch in a parallel manner as this implementation was not discussed in the original BinarySearch paper and is not the focus of our paper.

In [19]:
%pip install tabulate
from tabulate import tabulate
mydata = [
    ["BinarySearch", 
     f"{np.array(results['group3']['binary']).mean()}", 
     f"{np.array(results['group100']['binary']).mean()}", 
     f"{np.array(results['individual1000']['binary']).mean()}"],
    ["SCF",
     f"{np.array(results['group3']['scf']).mean()}", 
     f"{np.array(results['group100']['scf']).mean()}", 
     f"{np.array(results['individual1000']['scf']).mean()}"]
]
head = ["Method", "Group-3", "Group-100", "Individual-1000"]
print(tabulate(mydata, headers=head, tablefmt="grid"))

Note: you may need to restart the kernel to use updated packages.
+--------------+-----------+-------------+-------------------+
| Method       |   Group-3 |   Group-100 |   Individual-1000 |
+==============+===========+=============+===================+
| BinarySearch |   1.68895 |     52.4976 |          597.821  |
+--------------+-----------+-------------+-------------------+
| SCF          |  13.1107  |     13.3182 |           14.0967 |
+--------------+-----------+-------------+-------------------+
